<center><a href="https://www.featuretools.com/"><img src="img/featuretools-logo.png" width="400" height="200" /></a></center>

<h2> A Featuretools Baseline </h2>
<p>The following tutorial illustrates a featuretools baseline model for the NYC Taxi Trip Duration competition on Kaggle. This notebook follows the structure of the previous worksheet, but uses deep feature synthesis to create the model.</p>

<h2>Step 1: Download raw data </h2>
<p>As always, if you have not yet downloaded the data it can be found at the <a href="https://www.kaggle.com/c/nyc-taxi-trip-duration/data">Kaggle website</a>. After installing featuretools following <a href = "https://docs.featuretools.com/">the instructions in the documentation</a> you can run the following.
</p>


In [1]:
import pandas as pd
import numpy as np
import featuretools as ft
import utils
ft.__version__

'0.1.8'

In [2]:
TRAIN_DIR = "data/train.csv"
TEST_DIR = "data/test.csv"

data_train, data_test = utils.read_data(TRAIN_DIR, TEST_DIR)

data_train.head(5)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,False,455
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,False,663
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,False,2124
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,False,429
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,False,435


<h2>Step 2: Prepare the Data </h2>
<p>Let's create another column to define test and train datasets.</p>

In [3]:
data_train['test_data'] = False
data_test['test_data'] = True

<p>We can now combine the data. </p>

In [4]:
data = pd.concat([data_train, data_test])

<h2>Step 3: Create baseline features using DFS </h2>
<p>Instead of manually creating features, such as month of <b>pickup_datetime</b>, we can let featuretools come up with them. </p>

<p>Within featuretools there is a standard format for representing data that is used to set up predictions and build features. A <b>EntitySet</b> stores information about entities (database table), variables (columns in database tables), relationships, and the data itself. </p>

<p> First, we create the EntitySet.</p>

In [5]:
es = ft.EntitySet("taxi")

<p>We can then use the `entity_from_dataframe` method to add an Entity called <i>trips</i>. We want to track the `id`, the `time_index` and specify other types of variables we care about in this entity. </p>

<p>As a note: Featuretools will try to interpret the types of variables. We can override this interpretation by specifying the types. In this case, I wanted <b>passenger_count</b> to be a type of Ordinal, and <b>vendor_id</b> to be of type Categorical.</p>

In [6]:
from featuretools import variable_types as vtypes

trip_variable_types = {
    'passenger_count': vtypes.Ordinal, 
    'vendor_id': vtypes.Categorical,
}

es.entity_from_dataframe(entity_id="trips",
                         dataframe=data,
                         index="id",
                         time_index='pickup_datetime',
                         variable_types=trip_variable_types)

Entityset: taxi
  Entities:
    trips (shape = [2050266, 11])
  Relationships:
    No relationships

<p>We can also normalize some of the columns to create new entities. So a <i>vendors</i> entity is created based on the unique values in the <i>vendor_id</i> column in <i>trips</i>.</p>

In [7]:
es.normalize_entity(base_entity_id="trips",
                    new_entity_id="vendors",
                    index="vendor_id")

es.normalize_entity(base_entity_id="trips",
                    new_entity_id="passenger_cnt",
                    index="passenger_count")

Entityset: taxi
  Entities:
    vendors (shape = [2, 2])
    trips (shape = [2050266, 11])
    passenger_cnt (shape = [8, 2])
  Relationships:
    trips.vendor_id -> vendors.vendor_id
    trips.passenger_count -> passenger_cnt.passenger_count

<p>We can specify the time for each instance of the target_entity to calculate features. The timestamp represents the last time data can be used for calculating features by DFS. This is specified using a dataframe of cutoff times. Below we can see that the cutoff time for each trip is the pickup time.</p>

In [8]:
cutoff_time = (es['trips'].df[['id', 'pickup_datetime']])

<p>Given this dataset, we would have about 2 million unique cutoff times. This is a good use case to use the approximate features parameter of DFS. In a large dataset, direct features that are aggregations on the prediction entity may not change much from cutoff time to cutoff time. Calculating the aggregation features at specific times every hour and using it for all cutoff times within the hour would save time and perhaps not lose much information. The approximate parameter in DFS lets you specify a window size to use when approximating these direct aggregation features.</p>

<p>We now create features using DFS.</p>

<b>Note, we can use an already calculated feature_matrix by doing the following:</b>
<p>You must copy and run the code.</p>
```python
feature_matrix = pd.read_csv('https://s3.amazonaws.com/featuretools-static/nyc_taxi/fm_simple.csv', 
                             index_col='id')
features = feature_matrix.columns.values
```

In [9]:
from featuretools.primitives import (Day, Hour, Minute, Month, Weekday, Week, Weekend)

es.add_interesting_values()

trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, Weekend]

feature_matrix, features = ft.dfs(entityset=es,
                                  target_entity="trips",
                                  trans_primitives=trans_primitives,
                                  drop_contains=['trips.test_data'],
                                  verbose=True,
                                  cutoff_time=cutoff_time,
                                  approximate='36d')

Building features: 72it [00:00, 322.72it/s]
Progress: 100%|██████████| 6/6 [05:20<00:00, 53.32s/cutoff time]


<p>Here are the features created. Notice how some of the features match the manually created features in the previous notebook.</p>

In [10]:
print len(features)

40


In [11]:
features[:25]

[<Feature: passenger_count>,
 <Feature: dropoff_longitude>,
 <Feature: store_and_fwd_flag>,
 <Feature: vendor_id>,
 <Feature: test_data>,
 <Feature: pickup_latitude>,
 <Feature: pickup_longitude>,
 <Feature: trip_duration>,
 <Feature: dropoff_latitude>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: passenger_cnt.SUM(trips.pickup_longitude)>,
 <Feature: passenger_cnt.WEEK(first_trips_time)>,
 <Feature: vendors.SUM(trips.dropoff_longitude)>,
 <Feature: vendors.DAY(first_trips_time)>,
 <Feature: passenger_cnt.WEEKDAY(first_trips_time)>,
 <Feature: vendors.SUM(trips.pickup_latitude)>,
 <Feature: passenger_cnt.SUM(trips.dropoff_latitude)>,
 <Feature: vendors.WEEKDAY(first_trips_time)>,
 <Feature: vendors.MONTH(first_trips_time)>]

<h2>Step 3: Build the Model </h2>

<p>We need to retrieve our labels for the train dataset, so we should merge our current feature matrix with the original dataset. </p>
<p>We also get the log of the trip duration so that a more linear relationship can be found.</p>

In [12]:
# separates the whole feature matrix into train data feature matrix, train data labels, and test data feature matrix 
X_train, labels, X_test = utils.get_train_test_fm(feature_matrix)
labels = np.log(labels.values + 1)

In [13]:
model = utils.train_xgb(X_train, labels)

[0]	train-rmse:5.00303	valid-rmse:5.00293
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 60 rounds.
[10]	train-rmse:0.996953	valid-rmse:0.99796
[20]	train-rmse:0.559007	valid-rmse:0.562018
[30]	train-rmse:0.500829	valid-rmse:0.504979
[40]	train-rmse:0.480289	valid-rmse:0.485395
[50]	train-rmse:0.462385	valid-rmse:0.468386
[60]	train-rmse:0.437639	valid-rmse:0.444571
[70]	train-rmse:0.431325	valid-rmse:0.439002
[80]	train-rmse:0.424981	valid-rmse:0.433295
[90]	train-rmse:0.420665	valid-rmse:0.429507
[100]	train-rmse:0.417436	valid-rmse:0.426779
[110]	train-rmse:0.415885	valid-rmse:0.425593
[120]	train-rmse:0.413628	valid-rmse:0.423836
[130]	train-rmse:0.412455	valid-rmse:0.423168
[140]	train-rmse:0.411819	valid-rmse:0.422747
[150]	train-rmse:0.407039	valid-rmse:0.418423
[160]	train-rmse:0.404392	valid-rmse:0.416234
[170]	train-rmse:0.402947	valid-rmse:0.415275
[180]	train-rmse:0.39991	valid-rmse:0.412

<h2>Step 4: Make a Submission </h2>

In [14]:
submission = utils.predict_xgb(model, X_test)
submission.head(5)

,trip_duration
id,
id0000002,1103.446045
id0000199,1986.509033
id0000446,1023.389771
id0000587,1326.767944
id0000604,193.377487


In [15]:
submission.to_csv('trip_duration_ft_simple.csv', index=True, index_label='id')

<dt>This solution:</dt>
<dd>&nbsp; &nbsp; Received a score of 0.45616 on the Kaggle competition.</dd>
<dd>&nbsp; &nbsp; Placed 565 out of 1082.</dd>
<dd>&nbsp; &nbsp; Beat 48% of competitors on the Kaggle competition.</dd>
<dd>&nbsp; &nbsp; Scored 2% better than the baseline solution</dd>
<dd>&nbsp; &nbsp; Had a modeling RMSLE of 0.40489</dd>

September 7, 2017.

<h2>Additional Analysis</h2>
<p>Let's look at how important each feature was for the model.</p>

In [16]:
feature_names = X_train.columns.values
ft_importances = utils.feature_importances(model, feature_names)
ft_importances[:20]

,feature_name,importance
6,dropoff_latitude,8141.0
1,dropoff_longitude,7465.0
4,pickup_latitude,6970.0
5,pickup_longitude,6001.0
8,HOUR(pickup_datetime),2976.0
7,DAY(pickup_datetime),2394.0
23,MINUTE(pickup_datetime),2251.0
25,WEEK(pickup_datetime),1665.0
9,WEEKDAY(pickup_datetime),1472.0
27,passenger_cnt.SUM(trips.pickup_longitude),1199.0
